# Create a database of operations archives

In [ ]:
import pandas as pd
import os
rarc_ops_db = '/home/sbf000/ss6/rarc_ops_db.csv'

if os.path.isfile(rarc_ops_db):
    print('reading database')
    operation_database_df = pd.read_csv(rarc_ops_db)
else:
    print('creating database')
    index = '/home/smco503/Master_Protocoles/protocole_operation'

    with open(index,'r') as fp:
        lines = fp.readlines()

    hall = 'PPP5'
    operation_database_df = pd.DataFrame([{'directory':f"{l.split()[7].replace('${archiveHall}',hall).strip()}/{l.split()[0].replace('&','').strip()}/",'id':l.split()[0].replace('&','').strip(), 'trunk':l.split()[4].strip(), 'branch':l.split()[5].strip()} for l in  lines if (l.split()[0][0] != '#') and (len(l.split())>=7)])
    # operation_database_df.sort_values('trunk')
    operation_database_df.to_csv('/home/sbf000/ss6/rarc_ops_db.csv', index=False)

operation_database_df

In [ ]:
pd.set_option('display.max_rows', None)
operation_database_df.loc[operation_database_df.branch.str.contains('glbgrib')].sort_values(['trunk','branch'])

# Get the available files for a branch

In [ ]:
import pandas as pd
import numpy as np
import os
import re

def get_files_for_branch(operation_database_df, trunk:str, branch:str) -> list:
    directory = operation_database_df.loc[operation_database_df.branch == branch].iloc[0].directory
    sources = np.unique([directory + f for f in os.listdir(directory) if re.search(r'[0-9]{6}$', f) and (f[0] != '.')])
    display([f for f in os.listdir(directory)])
    data = []
    for f in sources:
        with open(f,'r') as fp:
            data = data + fp.readlines()
    files = ','.join([l.split()[0].strip() for l in data])
    return files

    # # get the file names from the all the read files
    # files_df.loc[files_df.directory == directory,'files'] = ','.join([l.split()[0].strip() for l in data])



def get_min_max_dates(files):
    sorted_files = np.sort([f[0:10] for f in files.split(',')])
    # display(sorted_files)
    if (sorted_files.size > 0):
        if sorted_files[0] != '':
            min = pd.to_datetime(f'{sorted_files[0][0:8]}T{sorted_files[0][8:10]}')
            max = pd.to_datetime(f'{sorted_files[-1][0:8]}T{sorted_files[-1][8:10]}')
            return min,max
    return np.nan,np.nan



# files = get_files_for_branch(operation_database_df,'forecasts','lam.nat.pres')
# print(files)
# get_min_max_dates(files)


In [ ]:
sub_df = operation_database_df.loc[operation_database_df.trunk.str.contains('grib2')]
display(sub_df)
for i in sub_df.index:
    files = get_files_for_branch(sub_df,sub_df.at[i,'trunk'],sub_df.at[i,'branch'])
# files = get_files_for_branch(sub_df,'ncep','grib.sref')
# np.sort(files.split(','))
# [f for f in files.split(',') if '20200111' in f]

    
    min_date,max_date = (get_min_max_dates(files))
    sub_df.at[i,'min_date'] = min_date
    sub_df.at[i,'max_date'] = max_date

# sub_df.loc[~sub_df.min_date.isna()][['trunk','branch','min_date','max_date']].sort_values(by='trunk').reset_index(drop=True)
files

In [ ]:
sub_df.loc[~sub_df.min_date.isna()].sort_values('min_date')

In [ ]:
sub_df = operation_database_df.loc[operation_database_df.trunk.str.contains('observations') & (operation_database_df.branch.str.contains('grib'))]
sub_df
# files = get_files_for_branch(sub_df,'operations','reghyb')
# np.sort(files.split(','))

In [ ]:
cap = 48
for i in [0,6,12,18]:
    sum = i + 24
    while (sum <= cap) and (sum//24==1):
        print(sum)
    # for j in range(0,49):
    #     print(j)
        # if (i + j) < 48:
        #     # print((i+j)//24)
        #     if (i+j) == 24:
        #         print(f'{i:02d}_{j:03d}')
        # break


In [ ]:
sub_df = operation_database_df.loc[operation_database_df.branch.str.contains('hrdps')]
for i in sub_df.index:
    files = get_files_for_branch(sub_df,sub_df.at[i,'trunk'],sub_df.at[i,'branch'])
    # print(files)
    min_date,max_date = (get_min_max_dates(files))
    sub_df.at[i,'min_date'] = min_date
    sub_df.at[i,'max_date'] = max_date

sub_df.loc[~sub_df.min_date.isna()][['trunk','branch','min_date','max_date']].sort_values(by='trunk').reset_index(drop=True)

# rdsps.netcdf archive contents for the last 5 years

In [ ]:
# filter files on those hours
# filtered_files = [f for f in files if f[11:14] in ['000','024','030','036','042','048']]
filtered_files = files
# display('filtered files')
# display(filtered_files)
# display(f'{len(filtered_files)} available filtered files')

# find the unique dates
unique_dates = np.sort(np.unique([f'{f[0:8]}' for f in filtered_files.split(',')]))

# create a date_string for directives
unique_dates_str = [f'{d[0:4]},{d[4:6]},{d[6:8]},{d[0:4]},{d[4:6]},{d[6:8]}' for d in unique_dates]
# display('unique dates')
# display(unique_dates)


# create a dataframe of directives
df = pd.DataFrame(
    {
        'target':np.full_like(unique_dates,'/home/sbf000/ss6',dtype='<U16'), 
        'filter':np.full_like(unique_dates,'/home/sbf000/src/rarc/filter.sh', dtype='<U31') , 
        'postprocess':np.full_like(unique_dates,'nopost',dtype='<U6'), 
        'date':unique_dates_str,
        'branche':np.full_like(unique_dates,'operation.forecasts.lam.nat.pres',dtype='<U32'),
        'ext':np.full_like(unique_dates,np.nan), 
        'heure':np.full_like(unique_dates,np.nan), 
        'priority':np.full_like(unique_dates,'online',dtype='<U6'),
        'inc':np.full_like(unique_dates,'1',dtype='<U1'),
        'inc':np.full_like(unique_dates,'#',dtype='<U1'),
        'real_date':unique_dates,
    })

# convert date string to real datetime object for searching
df['search_date'] = pd.to_datetime(df['real_date'])

# fill the ext and heure columns of the dataframe
for d in unique_dates:
    df.loc[df.real_date==d,'ext'] = ','.join(np.sort(np.unique([f'{f[11:]}' for f in filtered_files if f[0:8] == d])).tolist())
    df.loc[df.real_date==d,'heure'] = ','.join(np.sort(np.unique([f'{f[8:10]}' for f in filtered_files if f[0:8] == d])).tolist())

# remove_real_date column    
df.drop(columns='real_date',inplace=True)

# create a 5 year date mask
mask = (df['search_date'] > '2017-1-1') & (df['search_date'] <= '2021-1-1')

# create a dataframe for that 5 year period
reduced_df = df.loc[mask].drop(columns='search_date')

len(reduced_df)

reduced_df
mydict = reduced_df.iloc[0].to_dict()

for k,v in mydict.items():
    print(f'{k} = {v}')